<a href="https://colab.research.google.com/github/Purple-Raining/Conversation-Summarizer/blob/main/Task1_Conversation_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required Libraries
!pip install --upgrade openai

import os, json, re
from getpass import getpass
from typing import List, Dict, Any, Tuple

print("Packages installed and basic imports ready.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.6/947.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.107.0
    Uninstalling openai-1.107.0:
      Successfully uninstalled openai-1.107.0
Packages installed and basic imports ready.


In [ ]:
# Secure key input
groq_key = getpass("Paste your GROQ API key here: ").strip()
os.environ["GROQ_API_KEY"] = groq_key
print("Key stored safely in runtime.")


Paste your GROQ API key here: ··········
Key stored safely in runtime.


In [ ]:
class ConversationManager:
    def __init__(self, summarizer = None, k = 3):
        """
        Parameters:
        - summarizer-> a function that takes a list of messages and returns a summary
        - k-> perform summarization after every k-th conversation
        """
        self.history = []  # stores tuples of (role, message)
        self.summarizer = summarizer
        self.k = k
        self.run_count = 0  # count conversations for periodic summarization
        self.summary = ""   # stores summarized history

    def add_message(self, role: str, message: str):
        """Add a new message to the conversation."""
        self.history.append((role, message))
        self.run_count += 1

        # Check if we need to summarize
        if self.summarizer and self.run_count % self.k == 0:
            self.summarize_history()

    def get_history(self, last_n: int = None, max_chars: int = None):
        """Return truncated history according to limits."""
        history_text = "\n".join([f"{r}: {m}" for r, m in self.history])

        # Truncate by last n messages
        if last_n is not None:
            history_text = "\n".join([f"{r}: {m}" for r, m in self.history[-last_n:]])

        # Truncate by max characters
        if max_chars is not None:
            history_text = history_text[-max_chars:]  # last max_chars characters

        return history_text

    def summarize_history(self):
        """Generate a summary and replace history with it."""
        if self.summarizer:
            self.summary = self.summarizer(self.history)
            # Reset history to summarized version
            self.history = [("summary", self.summary)]
            print(f"Summarized after {self.run_count} messages:\n{self.summary}\n")
